<a href="https://colab.research.google.com/github/ShadowRaccoon/PrograConcurrente2023C1/blob/main/TP1/TP1_Parte2_Java_M1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Desarrollo de la solución en Java**

Para la implementación en Java, decidimos crear una clase para las matrices que internamente tiene un vector de vectores. De esta manera, definimos las operaciones entre matrices, en este caso la multiplicación.
Decidimos implementar el patrón Strategy para modelar las distintas formas de multiplicar matrices, ya que este permite fácil ampliación y reusabilidad del código.

Decidimos utilizar Generics para no limitar la solución sólo a matrices de enteros.

Para poder generar números aleatorios, tuvimos que crear nuestra propia implementación, ya que Java no permite esto de forma nativa. En la misma utilizamos el patrón Singleton, para tener una única instancia del generador de números aleatorios que los genere todos con la misma semilla.

Para la solución concurrente, decidimos utilizar una expresión lambda que define una función anónima en lugar de las técnicas vistas en clase, ya que, en este caso, nos pareció excesivo y desprolijo crear una clase entera para multiplicar las filas de la matriz, ya que la misma no sería reutilizable. 

In [8]:
%%writefile Main.java
import java.text.MessageFormat;
import java.util.ArrayList;
import java.util.List;
import java.util.Random;
import java.util.concurrent.atomic.AtomicReference;

//Anidamos todas las clases dentro del Main porque colab no nos reconocía las clases en distintos archivos.
public class Main
{
  private final static int LOWER_LIMIT_RANDOM = -32;
  private final static int HIGHER_LIMIT_RANDOM = 32;
  private final static int MAX_MATRIX_SIZE = 20;
  private final static int MIN_MATRIX_SIZE = 5;

  public static void main(String[] args)
  {
    //Utilizamos esta clase para poder pasar el tamaño de la matriz por referencia.
    AtomicReference<Integer> sizeReference = new AtomicReference<>();
    boolean isValid = validateArgs(args, sizeReference);
    if (!isValid)
    {
      System.out.println(MessageFormat.format("Se debe indicar por argumento un numero entero entre {0} y {1} para el tamaño de la matriz", MIN_MATRIX_SIZE, MAX_MATRIX_SIZE));
      System.exit(1);
    }

    int matrixSize = sizeReference.get();

    //Creación e inicialización de las matrices.
    IntegerMatrix matrixA = new IntegerMatrix(matrixSize);
    matrixA.randomizeData(LOWER_LIMIT_RANDOM, HIGHER_LIMIT_RANDOM);

    IntegerMatrix matrixB = new IntegerMatrix(matrixSize);
    matrixB.randomizeData(LOWER_LIMIT_RANDOM, HIGHER_LIMIT_RANDOM);

    try
    {
      //Utilizamos la estrategia secuencial para la multiplicación de matrices.
      Matrix<Integer> matrixCS = matrixA.multiply(matrixB, new SecuencialMultiplicationStrategy());

      //Utilizamos la estrategia concurrente con hilos para la multiplicación de matrices.
      Matrix<Integer> matrixCH = matrixA.multiply(matrixB, new ConcurrentMultiplicationStrategy());

      //Mostramos las matrices generadas y resultantes por pantalla.
      printMatrix("A", matrixA.get());
      printMatrix("B", matrixB.get());
      printMatrix("CS", matrixCS.get());
      printMatrix("CH", matrixCH.get());
      boolean areEqual = matrixCS.equals(matrixCH);
      System.out.println(MessageFormat.format("Las matrices son {0}", areEqual ? "iguales" : "distintas"));
    } catch (Exception e)
    {
      System.out.println(e.getMessage());
    }

  }

  private static boolean validateArgs(final String[] args, AtomicReference<Integer> matrixSize)
  {
    boolean isValid = true;
    if (args.length >= 1)
    {
      try
      {
        //Leemos el tamaño de la matriz desde los argumentos.
        int parsedValue = Integer.parseInt(args[0]);
        if (parsedValue >= MIN_MATRIX_SIZE && parsedValue <= MAX_MATRIX_SIZE)
        {
          matrixSize.set(parsedValue);
        } else
        {
          isValid = false;
        }
      } catch (
          IllegalArgumentException ex) //Si no se puede convertir en entero lanza esta excepción.
      {
        isValid = false;
      }
    } else
    {
      isValid = false;
    }

    return isValid;
  }

  /**
   * Imprime la matriz con su alias.
   *
   * @param alias  nombre o alias de la matriz que se desea mostrar.
   * @param matrix matriz que se desea mostrar.
   */
  public static void printMatrix(final String alias, final Integer[][] matrix)
  {
    System.out.println(MessageFormat.format("---- Matriz {0} ----", alias));

    for (Integer[] row : matrix)
    {
      for (Integer element : row)
      {
        System.out.print("| " + String.format("%7s", element) + " ");
      }
      System.out.println("|");
    }
    System.out.println();
  }

  /**
   * Representa las matrices de cualquier tipo de dato y sus operaciones básicas.
   */
  public static abstract class Matrix<E>
  {
    protected final E[][] data;
    protected final int rows;
    protected final int columns;
    private final Operator<E> operator;

    /**
     * Crea matrices rectangulares inicializadas en cero.
     *
     * @param rows    la cantidad de filas de la matriz.
     * @param columns la cantidad de columnas de la matriz.
     * @throws IllegalArgumentException si {@code rows} o {@code columns} son menores o iguales que cero o si {@code data} o {@code operator} son nulos.
     */

    protected Matrix(int rows, int columns, E[][] data, Operator<E> operator)
    {
      if (rows <= 0 || columns <= 0)
      {
        throw new IllegalArgumentException("Debe especificar un numero entero positivo para las dimensiones de la matriz");
      }

      if (data == null || operator == null)
      {
        throw new IllegalArgumentException();
      }

      this.operator = operator;
      this.rows = rows;
      this.columns = columns;
      this.data = data;
    }

    public final int rows()
    {
      return rows;
    }

    public final int columns()
    {
      return columns;
    }


    /**
     * Obtiene el elemento almacenado en la posicion indicada.
     *
     * @param row    el indice basado en cero de la fila deseada.
     * @param column el indice basado en cero de la columna deseada.
     */
    public final E get(int row, int column) throws ArrayIndexOutOfBoundsException
    {
      if (row >= rows() || column >= columns())
      {
        throw new ArrayIndexOutOfBoundsException();
      }
      return data[row][column];
    }

    /**
     * Obtiene el array que conforma la fila indicada.
     *
     * @param row el indice basado en cero de la fila deseada.
     */
    public final E[] get(int row) throws ArrayIndexOutOfBoundsException
    {
      if (row >= rows())
      {
        throw new ArrayIndexOutOfBoundsException();
      }
      return data[row];
    }

    public final E[][] get()
    {
      return data;
    }

    /**
     * Inserta el valor en las fila y columna indicadas por parámetro.
     *
     * @param row    el índice basado en cero de la fila en que se debe insertar en la matriz.
     * @param column el índice basado en cero de la columna en que se debe insertar en la matriz.
     * @param value  el valor a insertar en la matriz.
     * @throws ArrayIndexOutOfBoundsException si {@code row} o {@code column} no están dentro de los límites de la matriz.
     */
    public final void set(int row, int column, E value) throws ArrayIndexOutOfBoundsException
    {
      if (row >= rows() || row < 0 || column >= columns() || column < 0)
      {
        throw new ArrayIndexOutOfBoundsException();
      }
      data[row][column] = value;
    }

    /**
     * @param row   el indice basado en cero de la fila en que se debe insertar en la matriz.
     * @param value la fila a insertar en la matriz.
     * @throws ArrayIndexOutOfBoundsException si {@code rows} no esta dentro de los limites de la matriz.
     */
    public final void set(int row, E[] value) throws ArrayIndexOutOfBoundsException
    {
      if (row >= rows() || row < 0)
      {
        throw new ArrayIndexOutOfBoundsException();
      }
      data[row] = value;
    }

    /**
     * Valida si las matrices son compatibles para la multiplicacion.
     * Compara si la cantidad de columnas de la primera es igual a la cantidad de filas de la segunda.
     *
     * @param matrixB la segunda matriz a comparar.
     */
    private boolean isCompatibleForMultiplication(Matrix<E> matrixB)
    {
      return this.columns() == matrixB.rows();
    }

    /**
     * Multiplica dos matrices.
     *
     * @param matrixB                la segunda matriz a multiplicar.
     * @param multiplicationStrategy la estrategia de multiplicacion a utilizar.
     * @throws IllegalArgumentException si las matrices no son compatibles.
     * @throws Exception                si ocurre algún error durante la multiplicación de matrices.
     */
    public Matrix<E> multiply(Matrix<E> matrixB, MultiplicationStrategy multiplicationStrategy) throws Exception
    {
      if (!this.isCompatibleForMultiplication(matrixB))
      {
        throw new IllegalArgumentException("Las matrices no son compatibles para su multiplicacion.");
      }

      Matrix<E> result = this.createInstance(this.columns(), matrixB.rows());

      return multiplicationStrategy.multiply(this, matrixB, result, operator);
    }

    protected abstract Matrix<E> createInstance(int columns, int rows);

    // Sobreescribimos el equals para poder comparar las matrices por contenido.
    @Override
    public boolean equals(Object other)
    {
      // Si es el mismo objeto, son iguales.
      if (other == this)
      {
        return true;
      }
      // Si no son de la misma clase son distintos.
      if (this.getClass() != other.getClass())
      {
        return false;
      }

      //Suprimimos el error porque en las anteriores nos aseguramos de que esta conversión es posible. 
      //A partir de open jdk 14 esto no es necesario porque se puede usar pattern matching, pero colab usa jdk 11.
      @SuppressWarnings("unchecked") 
      Matrix<E> otherMatrix = (Matrix<E>) other;

      // Si no tienen la misma cantidad de filas o de columnas son distintas
      if (this.rows() != otherMatrix.rows() || this.columns() != otherMatrix.columns())
      {
        return false;
      }

      // Comparamos por contenido.
      for (int i = 0; i < rows(); i++)
      {
        for (int j = 0; j < columns(); j++)
        {
          if (this.get(i, j) != otherMatrix.get(i, j))
          {
            return false;
          }
        }
      }

      // Si todo lo anterior no aplica, son iguales.
      return true;
    }
  }

  /**
   * Representa una matriz de enteros con sus posibles operaciones.
   */
  public static class IntegerMatrix extends Matrix<Integer>
  {
    /**
     * Crea matrices cuadradas inicializadas en cero.
     *
     * @param size la cantidad de filas y columnas de la matriz.
     * @throws IllegalArgumentException si {@code rows} o {@code columns} son menores o iguales que cero.
     */
    public IntegerMatrix(int size) throws IllegalArgumentException
    {
      this(size, size);
    }

    /**
     * Crea matrices rectangulares inicializadas en cero.
     *
     * @param rows    la cantidad de filas de la matriz.
     * @param columns la cantidad de columnas de la matriz.
     * @throws IllegalArgumentException si {@code rows} o {@code columns} son menores o iguales que cero.
     */
    public IntegerMatrix(int rows, int columns) throws IllegalArgumentException
    {
      super(rows, columns, new Integer[rows][columns], Operators.integerOperator);
    }

    /**
     * Inicializa la matriz con valores aleatorios.
     *
     * @param minValue valor minimo que puede usarse.
     * @param maxValue valor maximo que puede usarse.
     */
    public void randomizeData(int minValue, int maxValue)
    {
      for (int i = 0; i < rows(); i++)
      {
        for (int j = 0; j < columns(); j++)
        {
          data[i][j] = RangeRandom.randomInt(minValue, maxValue);
        }
      }
    }

    @Override
    protected Matrix<Integer> createInstance(int rows, int columns)
    {
      return new IntegerMatrix(rows, columns);
    }
  }

  // Utilizamos el patrón Strategy que nos permite agregar facilmente distintas estrategias para multiplicar matrices.

  /**
   * Define la estructura de las estrategias de multiplicación.
   */
  public interface MultiplicationStrategy
  {

    <E> Matrix<E> multiply(Matrix<E> matrixA, Matrix<E> matrixB, Matrix<E> resultMatrix, Operator<E> operator) throws Exception;
  }

  /**
   * Implementación secuencial de las estrategias de multiplicación.
   */
  public static class SecuencialMultiplicationStrategy implements MultiplicationStrategy
  {
    @Override
    public <E> Matrix<E> multiply(Matrix<E> matrixA, Matrix<E> matrixB, Matrix<E> resultMatrix, Operator<E> operator)
    {
      for (int i = 0; i < matrixA.rows(); ++i)
      {
        for (int j = 0; j < matrixB.columns(); ++j)
        {
          resultMatrix.set(i, j, operator.getDefault());
          for (int k = 0; k < matrixB.rows(); ++k)
          {
            resultMatrix.set(i, j, operator.add(resultMatrix.get(i, j), operator.multiply(matrixA.get(i, k), matrixB.get(k, j))));
          }
        }
      }
      return resultMatrix;
    }
  }

  /**
   * Implementación concurrente con hilos de las estrategias de multiplicación.
   */
  public static class ConcurrentMultiplicationStrategy implements MultiplicationStrategy
  {
    @Override
    public <E> Matrix<E> multiply(Matrix<E> matrixA, Matrix<E> matrixB, Matrix<E> resultMatrix, Operator<E> operator) throws Exception
    {
      // Generamos e iniciamos los hilos.
      List<Thread> threadsExecuting = generateThreads(matrixA, matrixB, resultMatrix, operator);

      // Esperamos la finalizacion de todos los hilos.
      waitThreads(threadsExecuting);

      return resultMatrix;
    }

    private void waitThreads(List<Thread> threadsExecuting) throws InterruptedException
    {
      for (Thread thread : threadsExecuting)
      {
        thread.join();
      }
    }

    private <E> List<Thread> generateThreads(Matrix<E> matrixA, Matrix<E> matrixB, Matrix<E> resultMatrix, Operator<E> operator)
    {
      // Creamos una lista de hilos para no perder la referencia y poder esperarlos al terminar el proceso.
      List<Thread> threadsExecuting = new ArrayList<>();
      for (int i = 0; i < matrixA.rows(); i++)
      {
        final E[] currentRowData = matrixA.get(i);
        final int currentRowIndex = i;

        // Utilizamos una función anónima a modo de Runnable.
        Thread thread = new Thread(() -> multiplyRow(currentRowData, matrixB, resultMatrix, currentRowIndex, operator));
        thread.start();
        threadsExecuting.add(thread);
      }
      return threadsExecuting;
    }

    private <E> void multiplyRow(E[] row, Matrix<E> matrixB, Matrix<E> resultMatrix, int rowIndex, Operator<E> operator)
    {
      for (int i = 0; i < matrixB.columns(); i++)
      {
        resultMatrix.set(rowIndex, i, operator.getDefault());
        for (int j = 0; j < row.length; j++)
        {
          E currentValue = resultMatrix.get(rowIndex, i);
          E updatedValue = operator.add(currentValue, operator.multiply(row[j], matrixB.get(j, i)));
          resultMatrix.set(rowIndex, i, updatedValue);
        }
      }
    }
  }

  /**
   * Extiende las funcionalidades de Random para permitir la generación de números aleatorios dentro de un rango.
   */
  public static class RangeRandom
  {
    // Utilizamos el patrón Singleton para contar con una única instancia del objeto random
    // que genere distintos números aleatorios con la misma semilla.
    public static final Random random = new Random(System.currentTimeMillis());

    public static int randomInt(int start, int end)
    {
      return random.nextInt((end - start) + 1) + start;
    }
  }

  /**
   * Define las posibles operaciones entre elementos.
   */
  public interface Operator<E>
  {
    E add(E firstItem, E secondItem);

    E multiply(E firstItem, E secondItem);

    E getDefault();

  }

  /**
   * Define las operaciones para los enteros.
   */
  public static class IntegerOperator implements Operator<Integer>
  {
    @Override
    public Integer add(Integer firstItem, Integer secondItem)
    {
      return firstItem + secondItem;
    }

    @Override
    public Integer multiply(Integer firstItem, Integer secondItem)
    {
      return firstItem * secondItem;
    }

    @Override
    public Integer getDefault()
    {
      return 0;
    }
  }

  /**
   * Define los posibles operadores.
   */
  public static class Operators
  {
    public static final IntegerOperator integerOperator = new IntegerOperator();
  }

  /**
   * Define los posibles estrategias de multiplicación.
   */
  public static class MultiplicationStrategies
  {
    public static final MultiplicationStrategy secuencialMultiplicationStrategy = new SecuencialMultiplicationStrategy();
    public static final MultiplicationStrategy concurrentMultiplicationStrategy = new ConcurrentMultiplicationStrategy();
  }
}

Overwriting Main.java


## Pruebas de validación de parametros:

Probamos llamar al Main sin argumentos

In [9]:
!java Main.java

Se debe indicar por argumento un numero entero entre 5 y 20 para el tamaño de la matriz


Probamos llamar al Main con argumentos por debajo de los valores permitidos

In [10]:
!java Main.java 3 

Se debe indicar por argumento un numero entero entre 5 y 20 para el tamaño de la matriz


Probamos llamar al Main con argumentos por encima de los valores permitidos

In [11]:
!java Main.java 34

Se debe indicar por argumento un numero entero entre 5 y 20 para el tamaño de la matriz


## Ejecución correcta:

In [12]:
!java Main.java  7

---- Matriz A ----
|     -10 |      13 |     -24 |      18 |       0 |      28 |      27 |
|       2 |     -21 |     -11 |     -25 |      28 |      12 |      10 |
|     -18 |     -24 |     -29 |     -24 |     -18 |      -3 |      -9 |
|     -27 |      26 |     -10 |      31 |      15 |      27 |      -6 |
|      24 |      20 |      29 |       5 |     -20 |      -1 |      10 |
|       5 |     -20 |     -11 |      20 |      20 |     -25 |       4 |
|      27 |      18 |       6 |      11 |      11 |     -16 |      27 |

---- Matriz B ----
|      10 |     -15 |      -1 |     -24 |      11 |     -17 |       8 |
|      32 |     -21 |      -8 |     -30 |     -21 |      -6 |       8 |
|      12 |       0 |     -10 |     -22 |      11 |     -12 |     -25 |
|       6 |     -22 |     -21 |      11 |       5 |      23 |      22 |
|      14 |      27 |      29 |      28 |     -10 |     -22 |      16 |
|       9 |      29 |      -5 |      -7 |     -18 |      -2 |      32 |
|     -17 |       0 |    